In [ ]:
from RSoftSimulation import RSoftSim
from Circuit_Properties import *
from Functions import *
from HexProperties import *

simulation_val = {
    "launch_type": LaunchType.MM,
    "monitor_type": Monitor_Prop.FIBRE_MODE_POWER,
    "comp": Monitor_comp.MAJOR,
    "launch_tilt": 0,
    "launch_mode": '*',
    "launch_mode_radial": '*',
    "launch_random_set": 1,
    "num_paras": 144,
    "batch_num": 6,
}

with open("launch_config.json", "w") as launch_config:
    json.dump(simulation_val, launch_config, indent = 2)

# setup custom priors
custom_priors = {
            # "Taper_L": (1000.0, 60000.0),
            "taper": (5.0, 30) 
            }

sim = RSoftSim()
sim.init_priors(custom_priors)
sim.RunRSoft(True)


In [2]:
import pandas as pd
# Read the multiprocessing csv log into a DataFrame
df = pd.read_csv("best_params_log.csv", header=None)

param_columns = [prior_name for prior_name, (_,_) in custom_priors.items()] + ["Throughput"]

print("Best fitting values:")
for col in df.columns[1:]:
    print(f"{param_columns[col-1]}: {df[col].iloc[-1]}")

Best fitting values:
Taper_L: 60000.0
taper: 8.443874348918603
Throughput: 0.975194274
